In [ ]:
import cv2
import numpy
import pytesseract
from pathlib import Path
from PIL import Image
from corpus_unpdf import get_img, extract_slices

In [ ]:
PAGE_NUM = 0
im = get_img(Path().cwd() / "data" / "data1.pdf", pagenum=0)
opencvImage = cv2.cvtColor(numpy.array(im), cv2.COLOR_RGB2BGR)

In [ ]:
CASE_PARTIES_STRUCTURE = (1000,30)
# x <= 200 and w > 2000
# break on first

In [ ]:
JUSTICE_LIST = (20, 80)
# x > im_w / 2 and w > 300 and h > 300
# break on first

In [ ]:
base_image = opencvImage.copy()
im_h, im_w, im_d = opencvImage.shape
opencvImage.shape

In [ ]:
gray = cv2.cvtColor(opencvImage, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, JUSTICE_LIST)
dilate = cv2.dilate(thresh, kernel, iterations=1)
cv2.imwrite("temp/sample_dilated.png", dilate)

In [ ]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[1])

In [ ]:
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if x > im_w / 2 and w > 300 and h > 300:
        print(f"{x=}, {y=}, {w=}, {h=}")
        cv2.rectangle(img=opencvImage, pt1=(x,y), pt2=(x+w, y+h), color=(36, 255, 12), thickness=2)
        roi = base_image[y + h:im_h, 0 : x + im_w]
        break
cv2.imwrite("temp/sample_boxes.png", opencvImage)